In [ ]:
# ML related
from time import sleep, thread_time
import glob
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np

# Model training
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# system requirements
import os, sys


# Load dataset
path = os.getcwd()
all_files = glob.glob(os.path.join(path, "./labeled/*_label.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# Drop columns
df = df.drop(columns=["timestamp"]) # drop timestamp column
class_x = df.drop(columns=["class"])
class_y = df["class"]

# Split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(
    class_x,
    class_y,
    test_size=0.3, # 70% training and 30% test
    random_state=1,
)

df_analysis = pd.concat([x_test, y_test], axis=1) # this print the test data with the class
print(df["class"].value_counts())

In [ ]:
# 1. RandomForest Model Test Results
import pickle
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

random_forest = OneVsRestClassifier(RandomForestClassifier(random_state=1))

random_forest.fit(x_train, y_train)


In [ ]:
# 2. GB Model Test Results

from sklearn.ensemble import GradientBoostingClassifier

gb = OneVsRestClassifier(
    GradientBoostingClassifier(
        random_state=1,
        n_estimators=100,
        max_depth=10,
    )
)
gb.fit(x_train, y_train)


In [ ]:
# Logistic Regression Model Test Results
from sklearn.linear_model import LogisticRegression

logistic = OneVsRestClassifier(
    LogisticRegression(
        random_state=1,
        max_iter=10000,
        solver="lbfgs",
    )
)
logistic.fit(x_train, y_train)


In [ ]:
# Naive Bayes Model Test Results
from sklearn.naive_bayes import GaussianNB

naive_bayes = OneVsRestClassifier(
    GaussianNB()
)
naive_bayes.fit(x_train, y_train)


In [ ]:
# 3. Stacking Model Test Results
# change final_estimator to minimum 2 models

from sklearn.ensemble import StackingClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.neural_network import MLPClassifier

stacking = OneVsRestClassifier(
    StackingClassifier(
        estimators=[
            ("rf", random_forest),
            ("gb", gb),
            ("logistic", logistic),
            ("naive_bayes", naive_bayes),
        ],
        final_estimator=MLPClassifier(
            random_state=1,
            max_iter=10000,
            hidden_layer_sizes=(100, 100),
            solver="lbfgs",
        ),
    )
)
stacking.fit(x_train, y_train)


In [ ]:
import time, warnings
from turtle import color, title

import pickle  # dump model
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

FIG_SIZE = 7.5


def plot_confusion_matrix(y_test, y_pred, title, labels, color, normalize=False):
    if normalize == False:
        ConfusionMatrixDisplay.from_predictions(
            y_test,
            y_pred,
            labels=labels,
            cmap=color,
            xticks_rotation="vertical",
        )

        return
    ConfusionMatrixDisplay.from_predictions(
        y_test,
        y_pred,
        labels=labels,
        cmap=color,
        normalize="true",
        xticks_rotation="vertical",
    )
    return
    # cm.plot()


models = []
models.append(("rf", random_forest))
models.append(("naive_bayes", naive_bayes))
models.append(("stacking", stacking))
models.append(("logistic", logistic))
models.append(("gb", gb))


models_name = [
    "randomForest",
    "naive_bayes",
    "stacking",
    "logistic",
    "gradientBoost",
]
labels = ["Normal", "Command Injection TL", "Command Injection TH", "Dos"]

model_test_accuracy = []
model_test_precision = []
model_test_recall = []
model_test_f1 = []


for x, y in models:
    train_start = time.time()
    y.fit(x_train, y_train)
    train_finished = time.time()
    pred_start = time.time()
    y_pred = y.predict(x_test)
    pred_stop = time.time()
    y_pred_proba = y.predict_proba(x_test)[:, 1]
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
    precision = sklearn.metrics.precision_score(y_test, y_pred, average="macro")
    recall = sklearn.metrics.recall_score(y_test, y_pred, average="macro")
    confusion_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
    f1 = sklearn.metrics.f1_score(
        y_test, y_pred, average="macro", labels=np.unique(y_pred)
    )

    # append algorithm overall test results to list
    model_test_accuracy.append(accuracy)
    model_test_precision.append(precision)
    model_test_recall.append(recall)
    model_test_f1.append(f1)
    sklearn.metrics.multilabel_confusion_matrix(y_test, y_pred)

    # dump model
    pickle.dump(y, open("models/" + x + ".pkl", "wb"))

    print(
        """
        Model: %s 
        Dataset size: %s
        Accuracy: %f
        Precision: %f
        Recall: %f
        F1: %f
        Train time: %s
        Predict time: %s
        """
        % (
            x,
            len(df),
            accuracy,
            precision,
            recall,
            f1,
            train_finished - train_start,
            pred_stop - pred_start,
        )
    )

    # overall plot_confusion_matrix
    plot_confusion_matrix(
        y_test,
        y_pred,
        x,
        labels,
        plt.cm.Blues,
    )

# plot dash graph
fig, axes = plt.subplots(figsize=(FIG_SIZE, FIG_SIZE))
axes.title.set_text("Accuracy of Models")
axes.plot(
    models_name,
    model_test_accuracy,
    label="Accuracy",
    marker="o",
    color="blue",
    linestyle="dashed",
)


# plot histogram
fig, axes = plt.subplots(figsize=(FIG_SIZE, FIG_SIZE))
axes.bar(models_name, model_test_accuracy)
axes.set_title("Accuracy")
axes.set_ylabel("Accuracy")
axes.set_xlabel("Algorithms")
axes.grid(True)
plt.show()


# float a pie chart
fig, axes = plt.subplots(figsize=(FIG_SIZE, FIG_SIZE))
axes.pie(
    model_test_accuracy,
    labels=models_name,
    autopct="%1.1f%%",
    startangle=90,
)
axes.grid(True)
axes.set_title("Accuracy Comparison")
plt.show()


In [ ]:
# get all confirm attacks

import glob, pandas as pd
import os

path = os.getcwd()
all_files = glob.glob(os.path.join(path, "./labeled/*_label.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)


count = 0
file_name = "anomaly.csv"

# remove if the file exists
if os.path.exists(file_name):
    os.remove(file_name)


# save to new file if condition is met
with open(file_name, "w") as f:
    f.write(
        str(list(df.columns)[1:][:-1])
        .replace("[", "")
        .replace("]", "")
        .replace("'", "")
        .replace(" ", ""),
    )
    f.write("\n")

for i in range(0, len(df)):

    if str(df["class"][i]) != "Normal":

        with open("anomaly.csv", "a") as f:
            f.write(
                str(df["tank_liquidlevel"][i])
                + ","
                + str(df["flowlevel"][i])
                + ","
                + str(df["bottle_liquidlevel"][i])
                + ","
                + str(df["motor_status"][i]),
            )
            f.write("\n")

            count += 1

print("total number of atacks from logs: ", count)


In [ ]:
cols = ["tank_liquidlevel", "flowlevel", "bottle_liquidlevel", "motor_status"]

warnings.filterwarnings("ignore")


def get_result(
    model_name, tank_liquidlevel, flowlevel, bottle_liquidlevel, motor_status
):
    model = pickle.load(open("models/" + model_name + ".pkl", "rb"))

    result = model.predict(
        np.array(
            [
                [
                    tank_liquidlevel,
                    flowlevel,
                    bottle_liquidlevel,
                    motor_status,
                ]
            ]
        )
    )
    return result


# import csv file

count = 0
file_name = "anomaly.csv"
df = pd.read_csv(file_name)

# df.head()
# print(df["tank_liquidlevel"][3])

for i in range(0, len(df)):

    with open("all_test", "a") as f:
        print(
            get_result(
                "naive_bayes",
                float(df["tank_liquidlevel"][i]),
                float(df["flowlevel"][i]),
                float(df["bottle_liquidlevel"][i]),
                float(df["motor_status"][i]),
            )
        )
        f.write(
            str(
                get_result(
                    "naive_bayes",
                    float(df["tank_liquidlevel"][i]),
                    float(df["flowlevel"][i]),
                    float(df["bottle_liquidlevel"][i]),
                    float(df["motor_status"][i]),
                )
            )
            + "\n"
        )

    count += 1

# algorithm that works
"""
1. rf -> works
2. gb -> works
3. logistic -> works
4. nb -> works
5. stacking -> works
"""


print("total number of attacks detected by ml: ", count)
print("total number of attacks undetected by ml: ", len(df) - count)
